In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install pm4py

     |████████████████████████████████| 419kB 2.8MB/s 
     |████████████████████████████████| 40.6MB 115kB/s 
  Created wheel for ciso8601: filename=ciso8601-2.1.3-cp36-cp36m-linux_x86_64.whl size=28304 sha256=8a1ae8a177eb7f5c1c125b1b3765aeb45cd740a7323716bc5acfc8981bfd3707
  Stored in directory: /root/.cache/pip/wheels/eb/32/e4/13bdaf7e245f82667b21e0cfb03d21224691a47fa9f9bc80a6
Successfully built ciso8601


In [0]:
!ls '/content/drive/My Drive/Professor/diabetes_logs'

diabetes_log_10_test.xes  diabetes_log_10_train.xes


In [0]:
#Importing library to read log file
from pm4py.objects.log.importer.xes import factory as xes_import_factory
from pm4py.objects.log.exporter.csv import factory as csv_exporter
from pm4py.objects.conversion.log import factory as conversion_factory
from pm4py.objects.log.exporter.xes import factory as xes_exporter
from pm4py.algo.filtering.pandas.variants import variants_filter

from datetime import datetime
import pandas as pd
import collections
import operator
import re

In [0]:
#Reading XES fileafter 

filename = '/content/drive/My Drive/Professor/diabetes_logs/diabetes_log_10_train.xes'
log = xes_import_factory.apply(filename)

#Converting XES to CSV file
fileName = '/content/drive/My Drive/Professor/diabetes_logs/diabetes_log_10_train.csv'

csv_exporter.export(log, fileName)

# Reading the Converted CSV file
df_clean = pd.read_csv(fileName)

# Filtering only three columns
#df_clean = df_clean.loc[:,['case:concept:name','concept:name','time:timestamp']]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Reading dataset 

In [0]:
df_clean.head(5)

case:concept:name         concept:name             time:timestamp
0                117  Creatinine_abnormal  2133-03-21 21:25:00+00:00
1                117     Glucose_abnormal  2133-03-21 21:25:00+00:00
2                117     Glucose_abnormal  2133-03-22 01:45:00+00:00
3                117  Creatinine_abnormal  2133-03-22 01:45:00+00:00
4                117     Glucose_abnormal  2133-03-23 02:55:00+00:00

In [0]:
df_clean = df_clean.loc[:,['case:concept:name','concept:name','time:timestamp']]

In [0]:
# Counting how many activity are in this log file
c = 0
for i in df_clean.loc[:,"concept:name"].unique():
    c +=1
print(c)

806


In [0]:
act_rename = ["A01","A02","A03","A04","A05","A06","A07","A08","A09"]
act_dic = {}
c = 0
for i in df_clean.loc[:,"concept:name"].unique():
  act_dic[i] = act_rename[c]
  c +=1

def activity_change(row):
  rename = act_dic[row]
  return rename
df_clean["concept:name"] = df_clean["concept:name"].apply(activity_change)

In [0]:
# The following function change time format
def time_change(row):
  
  if len(row) == 32:
    dateTime = row[:19]
    zone = row[26:]
    newTime = dateTime+zone
    return newTime
  else:
    return row

df_clean["time:timestamp"] = df_clean["time:timestamp"].apply(time_change)
df_clean["time:timestamp"] = pd.to_datetime(df_clean["time:timestamp"],)
df_clean["case:concept:name"] = df_clean["case:concept:name"].astype(str)

In [0]:
log = conversion_factory.apply(df_clean)
# Saving log into xes file
xes_file = '/content/drive/My Drive/Benchmarks/BPIC14_f/BPIC14_f_original.xes'
xes_exporter.export_log(log, xes_file)

### List of Unique activity

In [0]:
unique_activity = df_clean["concept:name"].unique()
act_rename = unique_activity   

In [0]:
unique_id = df_clean["case:concept:name"].unique()

### Number of distinct trace

In [0]:
# Save all trace id and activity belongs to a trace into dictionary
def activity(df_eve,unique_id):
  total_traces = []
  grouped =  df_eve.groupby('case:concept:name',)
  for i in unique_id:
    group_filter = grouped.get_group(i)['concept:name']
    group_filter  = group_filter.reset_index(drop=True)
    total_traces.append(group_filter)

  return total_traces

In [0]:
total_traces_list = activity(df_clean,unique_id)

In [0]:
len(total_traces_list)

41353

### Calculating frequency of each activity

In [0]:
grouped =  df_clean.groupby('case:concept:name',)
freq_activity = {}
for i in unique_id:
  group_filter = grouped.get_group(i)['concept:name']
  #counts_act = group_filter.value_counts()
  for a,c in  group_filter.value_counts().iteritems():
    if a not in freq_activity.keys():
      freq_activity[a] = c
    else:
      freq_activity[a] +=c

freq_activity

{'A01': 41353,
 'A02': 87644,
 'A03': 43974,
 'A04': 40767,
 'A05': 23085,
 'A06': 54098,
 'A07': 33496,
 'A08': 1634,
 'A09': 43434}

### All possible activity combinations  

In [0]:
# The following functions gets combination of events
def combination_of_activity(activity_name):
  # Combine two two activity and save into activity name
  Activity_Name = []
  for i in range(len(activity_name)):
    for c in range(len(activity_name)):
      if [activity_name[i],activity_name[c]] not in Activity_Name:
        Activity_Name.append([activity_name[i],activity_name[c]])
  
  return Activity_Name


In [0]:
Activity_combi = combination_of_activity(act_rename)

### Calculating frequency of each combination of activity

In [0]:
# Count the combination
# The function gets trace list and activity combination as input
def combination_counter(trace_list,activity_combination):
  # The combination of activity is key and value is number of times the combination comimg together
  counts_of_combination = {}
  
  # Each activity goes into the loop one by one 
  for trace_activity in trace_list:
  
    combination_of_observation = []
    # Creating the combination and save it into Activity name 
    for i in range(len(trace_activity)-1):
        combination_of_observation.append([trace_activity[i],trace_activity[i+1]])
        
    # Checking whether the resulted combination is presented or not in the activity combination list
    # When a resulted combination is presented in activity combination, the resulted combination is saved into 
    # counts of combination as key and number of times it is presented into the unique trace saved into 
    # counts of combination as value of it

    for c in combination_of_observation:
      if c in activity_combination:
        k =  repr(c)
        if k not in counts_of_combination.keys():
          counts_of_combination[k] = 1
        else:
          counts_of_combination[k] += 1
    
  return counts_of_combination
#CountsofComb = combination_counter(total_traces_list,Activity_combi)
#CountsofComb

In [0]:
CountsofComb = combination_counter(total_traces_list,Activity_combi)
CountsofComb

{"['A01', 'A02']": 17485,
 "['A01', 'A03']": 9473,
 "['A01', 'A06']": 4791,
 "['A01', 'A07']": 3713,
 "['A01', 'A09']": 5891,
 "['A02', 'A02']": 8918,
 "['A02', 'A03']": 21286,
 "['A02', 'A04']": 11645,
 "['A02', 'A05']": 5360,
 "['A02', 'A06']": 16883,
 "['A02', 'A07']": 9210,
 "['A02', 'A09']": 14342,
 "['A03', 'A02']": 15577,
 "['A03', 'A04']": 7324,
 "['A03', 'A05']": 3311,
 "['A03', 'A06']": 9291,
 "['A03', 'A07']": 4286,
 "['A03', 'A09']": 4185,
 "['A04', 'A05']": 14414,
 "['A05', 'A04']": 8085,
 "['A05', 'A08']": 1634,
 "['A06', 'A02']": 15572,
 "['A06', 'A03']": 9317,
 "['A06', 'A04']": 8710,
 "['A06', 'A06']": 8863,
 "['A06', 'A09']": 11636,
 "['A07', 'A02']": 9213,
 "['A07', 'A03']": 3898,
 "['A07', 'A04']": 5003,
 "['A07', 'A07']": 8002,
 "['A07', 'A09']": 7380,
 "['A09', 'A02']": 20879,
 "['A09', 'A06']": 14270,
 "['A09', 'A07']": 8285}

### Calculating probability of combination for (ab|a) and (ab|b)

In [0]:
# The following functions finds probability of each combination of activity given activity
def prob_of_activity(CountsofComb,freq_activity,threshold_value,df):
  # Dictionary "ab_a" to save the probability of each combination of activity given first activity
  # Dictionary "ab_b" to save the probability of each combination of activity given second activity
  # Dictionary "similar" to save the probability of "ab_a" and "ab_b" interaction
  ab_a = {}
  ab_b = {}
  similar = {}

  #sorted_d = dict( sorted(CountsofComb.items(), key=operator.itemgetter(1),reverse=True))

  # Finding the probability of each combination of activity given first activity
  for k , v in CountsofComb.items():
    if k[2:5] in freq_activity:
      prob = round((v / freq_activity[k[2:5]]),3)  
      if prob >= threshold_value:
        a = repr([k[2:5],k[9:12]])
        ab_a[a] = prob


  # Finding the probability of each combination of acitivity given second activity
  for k , v in CountsofComb.items():
    if k[2:5] in freq_activity:
      prob = round((v / freq_activity[k[9:12]]),3)
      if prob >= threshold_value:
        b = repr([k[2:5],k[9:12]])
        ab_b[b] = prob
     

  for i in ab_a.keys():
    if i in ab_b.keys():
      c = repr([i[2:5],i[9:12]])
      prob_add = ab_a[i] + ab_b[i]
      similar[c] = round(prob_add,2)

  similar = dict( sorted(similar.items(), key=operator.itemgetter(1),reverse=True))
  similar_keys = list(similar.keys())
  similar_values = similar.values()
  print( "Before reordering: ",similar)
  similar = reorder_activitylist(similar,df)
  print("After reodering: ", similar)
  return similar

### Reordering selected combination of activity

In [0]:
def reorder_activitylist(similar,d_f):
  prob_p = []
  prob_a = []
  for a, p in similar.items():
    prob_a.append(a)
    prob_p.append(p)


  counter=collections.Counter(prob_p)
  prob_morethanone = []
  for prob, n in counter.items():
    if n > 1:
      prob_morethanone.append(prob)
      
  # print(prob_morethanone)
  for p in prob_morethanone:
    n = 0
    while counter[p] != n:

      prob_activity = []
      prob_activity_key = []
      for k,v in enumerate(prob_p):
        if v == p:
          prob_activity.append(prob_a[k])
          prob_activity_key.append(k)
      
      #print(prob_activity)
      #print(prob_activity_key)
      max_value,index_ = frequncyOfposition(prob_activity,d_f)
      #print(index_)
      final_order_key, final_order_value = reorder_activity(prob_activity,prob_activity_key,index_)
      #print(final_order_key)
      #print(final_order_value)
      
      # Reordering for
      new_key = 0
      for k in prob_activity_key:
        prob_a[k] = final_order_value[new_key]
        new_key +=1
      break
  return prob_a

In [0]:
# The following find frequency of position of combination activity 
# When two or more more combination has equal probability
def frequncyOfposition(prob_activity,d_f):
  max_value = []
  index_ = []
  
  for l in prob_activity:
    max_value_i = []
    index_i = []
    df = indexCounterOfactivity (d_f, l)

    activity_counts = {}
    for a in df:

      if len(a) >2:
        a_ = []
        for a_i in range(0,len(a)-1):
          a_.append([a[a_i],a[a_i+1]])

        for i in a_:
          i = repr(i)
          if i not in activity_counts.keys():
            activity_counts[i] = 1
          else:
            activity_counts[i] += 1 

      else:
        a = repr(a)
        if a not in activity_counts.keys():
          activity_counts[a] = 1
        else:
          activity_counts[a] += 1 

    for k, v in activity_counts.items(): 
      if v == max(activity_counts.values()):
        max_value_i.append(v)
        index_i.append(k)
        
        
    if len(max_value_i)>1:
      final_order = reorder_activity1(max_value_i,index_i)  
      max_value.append(max_value_i[final_order[0]])
      index_.append(index_i[final_order[0]])
    else:
      max_value.append(max_value_i[0])
      index_.append(index_i[0])

  return max_value, index_

In [0]:
def reorder_activity(prob_activity,prob_activity_key,index_):
  
  # convert string data types to int
  index_new = []
  for i in index_:
    num = re.findall(r'\d+', i) 
    result = list(map(int, num))
    index_new.append(result)

  # It is taking the first position of each activity
  position_of_first_activity = {}
  for i in range(len(index_new)):
    position_of_first_activity[i]= index_new[i][0]

  # ascending order the position
  ascending_order_result = sorted(position_of_first_activity.values())

  # Final order the combiation activity key and combination activity
  final_order_key = []
  final_order_value = []
  for i in ascending_order_result:
    for k,v in position_of_first_activity.items():
      if i == v:
        final_order_key.append(prob_activity_key[k])
        final_order_value.append(prob_activity [k])
  
  
  return final_order_key, final_order_value

In [0]:
def indexCounterOfactivity(df_eve, activity_name):
  indexOf_combination_activity = []
  # Taking unique number of trace  
  unique_id = df_eve["case:concept:name"].unique()
  # Filter all unique trace one by one
  for i in unique_id:
    # Save the index of matched activity
    event_index = []
    # Filter one trace at a time
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    # Reset the index of the filtered trace
    row_filter  = row_filter.reset_index(drop=True)
    
    # The activity of the trace will be combared with desired combination activity
    # if it matches occured, the index of activity is saved into event index
    for event in range(len(row_filter.loc[:,'concept:name'])-1):
      # if The two activity is identical in the combination activity, the following condition will be excuted  
      if activity_name[2:5] == activity_name[9:12]:
        if (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
          # The index of the activity will be appended into event index
          event_index.append(event)
          event_index.append(event + 1)
          
      # If the two activity is not identical in the combination activity, the following will be excuted
      elif (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
        # The index of the activity will be appended into index
        event_index.append(event)
        event_index.append(event + 1)

    if len(event_index) !=0:
      indexOf_combination_activity.append(event_index)
    
  return indexOf_combination_activity

In [0]:
def reorder_activity1(max_value,index_):
  index_max_value = sorted(max_value,reverse=True)
  # convert string data types to int
  index_new = []
  for i in index_:
    num = re.findall(r'\d+', i) 
    result = list(map(int, num))
    index_new.append(result)


  # Finding the equal frequency of index and save into empty set
  # Finding index of all value
  max_value_new = set()
  final_index = []
  for i in range(len(index_max_value)-1):
    if index_max_value[i] == index_max_value[i+1]:
      max_value_new.add(i)
      final_index.append(i)
      max_value_new.add(i+1)
      final_index.append(i+1)
    else:
      final_index.append(i)
  
  if len(max_value_new)>0:
    # Compare position of equal frequency by finding minimum value of in the position 
    new_value = {}
    for i in range(list(max_value_new)[0],len(max_value_new)):
      new_value[i] = min(index_new[i])

    # the final_index will be updated
    index_max_value1 = sorted(list(new_value.values()))
    fi = 0
    for i in index_max_value1:
      for k,v in new_value.items():
         if i == v:
            final_index[list(max_value_new)[fi]] = k
            fi +=1

    return final_index


### Concatenate the event based on highest probability of combination

In [0]:
def event_dropper(df_eve, activity_name, new_activity_name):
  # Taking unique number of trace  
  unique_id = df_eve["case:concept:name"].unique()
  # Creating empty dataframe
  df_final = pd.DataFrame(columns=df_eve.columns)
  
  # Filter all unique trace one by one
  for i in unique_id:
    # Save the index of matched activity
    event_index = []
    # Filter one trace at a time
    row_filter = df_eve.loc[df_eve['case:concept:name']==i,:]
    # Reset the index of the filtered trace
    row_filter  = row_filter.reset_index(drop=True)
    
    # The activity of the trace will be combared with desired combination activity
    # if it matches occured, the index of activity is saved into event index
    for event in range(len(row_filter.loc[:,'concept:name'])-1):
      # if The two activity is identical in the combination activity, the following condition will be excuted  
      if activity_name[2:5] == activity_name[9:12]:
        if (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
          # The index of the activity will be appended into event index
          event_index.append(event)
          #event_index.append(event + 1)
          
      # If the two activity is not identical in the combination activity, the following will be excuted
      elif (activity_name[2:5] == row_filter.loc[event,'concept:name']) & (activity_name[9:12] == row_filter.loc[event + 1,'concept:name']):
        # The index of the activity will be appended into index
        event_index.append(event)
        row_filter.loc[event + 1,'concept:name']=row_filter.loc[event + 1,'concept:name'].replace(activity_name[9:12],new_activity_name)
       
    # The list of the saved index of activity will be dropped from the trace
    row_filter.drop(row_filter.index[event_index],inplace=True)
    # The resulting trace will be merged with new dataframe 
    df_final = pd.concat([df_final, row_filter],ignore_index=True)

  return df_final

### Removing self loop

In [0]:
def remove_self_loop(df,file_Num):
     
  same_act = []
  act_rename = df["concept:name"].unique()
  unique_id = df["case:concept:name"].unique()
  total_traces_list = activity(df,unique_id)
  Activity_combi = combination_of_activity(act_rename)
  CountsofComb1 = combination_counter(total_traces_list,Activity_combi)

  sorted_df = dict( sorted(CountsofComb1.items(), key=operator.itemgetter(1),reverse=True))
  for k , v in sorted_df.items():
    #print(k, ":", v)
    if k[2:5]==k[9:12]:
      same_act.append(k)
      #print(k, ":", v)
  
  if len(same_act) >0:
    new_a = same_act[0]
    df1 = event_dropper(df, new_a, None)
    for a in range(1,len(same_act)):
      new_ac = same_act[a]
      df1 = event_dropper(df1,new_ac,None)
      
    csv_file = '/content/drive/My Drive/Benchmarks/BPIC14_f/AfterRemovingSelfLoop_{}.csv'.format(file_Num)
    df1.to_csv(csv_file,index=False)
    # Converting csv file to log file
    log = conversion_factory.apply(df1)
    # Saving log into xes file
    xes_file = '/content/drive/My Drive/Benchmarks/BPIC14_f/BPIC14_f_{}.xes'.format(file_Num)
    xes_exporter.export_log(log, xes_file)

  else:
    csv_file = '/content/drive/My Drive/Benchmarks/BPIC14_f/AfterRemovingSelfLoop_{}.csv'.format(file_Num)
    df.to_csv(csv_file,index=False)
    # Converting csv file to log file
    log = conversion_factory.apply(df)
    # Saving log into xes file
    xes_file = '/content/drive/My Drive/Benchmarks/BPIC14_f/BPIC14_f_{}.xes'.format(file_Num)
    xes_exporter.export_log(log, xes_file)


In [0]:
unique_id = df_clean["case:concept:name"].unique()
total_traces_list = activity(df_clean,unique_id)
Activity_combi = combination_of_activity(act_rename)
CountsofComb = combination_counter(total_traces_list,Activity_combi)

### Feed different cut off probability to get number of XES files

In [0]:
import numpy as np

threshold_values = np.arange(0.1, 1, 0.1)
n = 9
replace_activity = {}

for t_v in threshold_values:
  df = df_clean
  similar_activity = prob_of_activity(CountsofComb,freq_activity,0.9,df_clean)

  if len(similar_activity)>=1:
    # Remaining combination
    for i in range(len(similar_activity)):
      if similar_activity[i][2:5] == similar_activity[i][9:12]:
        continue
      else:
        df = event_dropper(df, similar_activity[i],"BB{}".format(i))
        for ak in [similar_activity[i][2:5], similar_activity[i][9:12]]:
          if ak not in replace_activity.keys():
            replace_activity[ak] = "BB{}".format(i)
            
    #for rk in replace_activity.keys():
      #df["concept:name"] = df["concept:name"].apply(lambda x: re.sub( rk, replace_activity[rk] , x))
    
    csv_file = '/content/drive/My Drive/Benchmarks/BPIC14_f/BeforerRemovingSelfLoop_{}.csv'.format(n)
    df.to_csv(csv_file,index=False)
    remove_self_loop(df,n)
      
  else:
    csv_file = '/content/drive/My Drive/Benchmarks/BPIC14_f/BeforerRemovingSelfLoop_{}.csv'.format(n)
    df.to_csv(csv_file,index=False)
    remove_self_loop(df,n)
  break
  n +=1

Before reordering:  {}
After reodering:  []
